# Setup

In [1]:
import json
import requests
from datetime import datetime as dt
from collections import defaultdict
import pandas as pd

In [2]:
#https://pypi.org/project/gamma-viewer/
from gamma_viewer import GammaViewer
from IPython.display import display, Markdown

In [3]:
def printjson(j):
    print(json.dumps(j,indent=4))
def print_json(j):
    printjson(j)

In [4]:
def post(name,url,message,params=None):
    if params is None:
        response = requests.post(url,json=message)
    else:
        response = requests.post(url,json=message,params=params)
    if not response.status_code == 200:
        print(name, 'error:',response.status_code)
        print(response.json())
        return {}
    return response.json()

def automat(db,message):
    automat_url = f'https://automat.renci.org/{db}/query'
    response = requests.post(automat_url,json=message)
    print(response.status_code)
    return response.json()

def strider(message):
    url = 'https://strider.renci.org/query?log_level=DEBUG'
    strider_answer = post(strider,url,message)
    return strider_answer

def aragorn(message, coalesce_type='xnone'):
    if coalesce_type == 'xnone':
        answer = post('aragorn','https://aragorn.renci.org/query',message)
    else:
        answer = post('aragorn','https://aragorn.renci.org/query',message, params={'answer_coalesce_type':coalesce_type})
    return answer

##

def bte(message):
    url = 'https://api.bte.ncats.io/v1/query'
    return post(strider,url,message)

def coalesce(message,method='all'):
    url = 'https://answercoalesce.renci.org/coalesce/graph'
    return post('AC'+method,url,message)

def striderandfriends(message):
    strider_answer = strider(message)    
    coalesced_answer = post('coalesce','https://answercoalesce.renci.org/coalesce/all',strider_answer)
    omni_answer = post('omnicorp','https://aragorn-ranker.renci.org/omnicorp_overlay',coalesced_answer)
    weighted_answer = post('weight','https://aragorn-ranker.renci.org/weight_correctness',omni_answer)
    scored_answer = post('score','https://aragorn-ranker.renci.org/score',weighted_answer)
    return strider_answer,coalesced_answer,omni_answer,weighted_answer,scored_answer

In [42]:
def print_errors(strider_result):
    errorcounts = defaultdict(int)
    for logmessage in strider_result['logs']:
        if logmessage['level'] == 'ERROR':
            jm = json.loads(logmessage['message'])
            words = jm['error'].split()
            e = " ".join(words[:-5])
            errorcounts[e] += 1
    for error,count in errorcounts.items():
        print(f'{error} ({count} times)')
        
def print_queried_sources(strider_result):
    querycounts = defaultdict(int)
    for logmessage in strider_result['logs']:
        j = json.loads(logmessage['message'])
        if 'url' in j:
            querycounts[j['url']] += 1
    for url,count in querycounts.items():
        print(f'{url} ({count} times)')
        
def print_query_for_source(strider_result,url):
    for logmessage in strider_result['logs']:
        j = json.loads(logmessage['message'])
        if 'url' in j and j['url'] == url:
            print(j)
            return j

In [6]:
def retrieve_ars_results(mid):
    message_url = f'https://ars.transltr.io/ars/api/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    results = {}
    for child in j['children']:
        if child['actor']['agent'] in ['ara-aragorn', 'ara-aragorn-exp']:
            childmessage_id = child['message']
            child_url = f'https://ars.transltr.io/ars/api/messages/{childmessage_id}'
            child_response = requests.get(child_url).json()
            try:
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except:
                nresults=0
            print( child['status'], child['actor']['agent'],nresults )
    return results

In [7]:
def get_provenance(message):
    """Given a message with results, find the source of the edges"""
    prov = defaultdict(lambda: defaultdict(int)) # {qedge->{source->count}}
    results = message['message']['results']
    kg = message['message']['knowledge_graph']['edges']
    edge_bindings = [ r['edge_bindings'] for r in results ]
    for bindings in edge_bindings:
        for qg_e, kg_l in bindings.items():
            for kg_e in kg_l:
                for att in kg[kg_e['id']]['attributes']:
                    if att['name'] == 'provenance':
                        source = att['value']
                        prov[qg_e][source]+=1
    qg_edges = []
    sources = []
    counts = []
    for qg_e in prov:
        for source in prov[qg_e]:
            qg_edges.append(qg_e)
            sources.append(source)
            counts.append(prov[qg_e][source])
    prov_table = pd.DataFrame({"QG Edge":qg_edges, "Source":sources, "Count":counts})
    return prov_table

## Query Specific

In [8]:
standup_json='StandupDefinitions/standup_2.json'

In [9]:
with open(standup_json,'r') as jsonfile:
    standup_info = json.load(jsonfile)

In [10]:
display(Markdown(f"# {standup_info['Query Title']}"))
display(Markdown(f"{standup_info['Query Description']}"))
print(f'Github Issue: {standup_info["github_issue"]}')

# 2. Chemicals to Gene

What chemicals are associated with SMS (UniProtKB:P52788)

Github Issue: https://github.com/NCATSTranslator/testing/issues/10


The query as run through the ARS:

In [11]:
query = json.loads(requests.get(standup_info['query_location']).content)
printjson(query)

{
    "message": {
        "query_graph": {
            "nodes": {
                "n0": {
                    "id": "UniProtKB:P52788",
                    "category": "biolink:Gene"
                },
                "n1": {
                    "category": "biolink:ChemicalSubstance"
                }
            },
            "edges": {
                "e01": {
                    "subject": "n0",
                    "object": "n1"
                }
            }
        }
    }
}


## ARS Assessment

In [12]:
ARS_Responses = [(dt.strptime(x['ARS_result_date'],'%Y-%m-%d'),x['ARS_result_id']) for x in standup_info['ARS_Results']]
ARS_Responses.sort()

In [13]:
for ars_date, ars_id in ARS_Responses:
    display(Markdown(f'### {ars_date}'))
    _ = retrieve_ars_results(ars_id)
    print(f'https://arax.ncats.io/?source=ARS&id={ars_id}')

### 2021-03-02 00:00:00

Done ara-aragorn 63
Done ara-aragorn-exp 0
https://arax.ncats.io/?source=ARS&id=d187edcd-f4a1-42e8-9248-eee0cb161fdb


### 2021-03-16 00:00:00

Done ara-aragorn 119
Done ara-aragorn-exp 0
https://arax.ncats.io/?source=ARS&id=5ae7b3a8-ab4d-4282-a027-80a7116f1265


### 2021-03-19 00:00:00

Done ara-aragorn 128
Done ara-aragorn-exp 86
https://arax.ncats.io/?source=ARS&id=0f16dbfa-444a-409f-a7b2-9d11506b0d1c


### 2021-03-25 00:00:00

Done ara-aragorn 128
Done ara-aragorn-exp 0
https://arax.ncats.io/?source=ARS&id=8675552b-34b1-46eb-a4f0-3e34179a1e12


## Strider Direct

In [14]:
start = dt.now()
strider_result = strider(query)
end = dt.now()
print(f"Strider produced {len(strider_result['message']['results'])} results in {end-start}.")

Strider produced 37 results in 0:01:45.849240.


### Provenance

In [15]:
prov = get_provenance(strider_result)
display(prov)

,QG Edge,Source,Count
0,e01,https://translator.broadinstitute.org/molepro/...,37


### Queried sources

In [16]:
print_queried_sources(strider_result)

https://automat.renci.org/ctd/reasonerapi (75 times)
https://automat.renci.org/mychem/reasonerapi (16 times)
https://automat.renci.org/pharos/reasonerapi (8 times)
https://stars-app.renci.org/cam-kp/query (18 times)
https://automat.renci.org/cord19-scibite/reasonerapi (2 times)
https://translator.broadinstitute.org/molepro/trapi/v1.0/query (2 times)
https://api.bte.ncats.io/v1/query (3 times)
https://automat.renci.org/hmdb/reasonerapi (2 times)
https://automat.renci.org/gtopdb/reasonerapi (1 times)


### Errors

In [17]:
print_errors(strider_result)

504 Server Error: Gateway Time-out for url: https://automat.renci.org/kegg/reasonerapi (2 times)
504 Server Error: Gateway Time-out for url: https://automat.renci.org/cord19-scigraph/reasonerapi (4 times)
504 Server Error: Gateway Time-out for url: https://automat.renci.org/chembio/reasonerapi (2 times)
429 Client Error: Too Many Requests for url: https://api.bte.ncats.io/v1/query (121 times)


### Results

In [18]:
view = GammaViewer(props={"data":strider_result})
display(view)

### Strider Assessment

Enter Assessment Here

In [23]:
print_query_for_source(strider_result,'https://automat.renci.org/pharos/reasonerapi')

{'message': 'Received response from KP', 'url': 'https://automat.renci.org/pharos/reasonerapi', 'request': {'message': {'query_graph': {'nodes': {'n0': {'category': 'biolink:Gene', 'is_set': False, 'id': 'NCBIGene:6611'}, 'n1': {'category': 'biolink:ChemicalSubstance', 'is_set': False}}, 'edges': {'e01': {'subject': 'n0', 'object': 'n1', 'predicate': 'biolink:directly_interacts_with'}}}}}, 'response': {'message': {'query_graph': {'nodes': {'n0': {'id': 'NCBIGene:6611', 'category': 'biolink:Gene', 'is_set': False}, 'n1': {'id': None, 'category': 'biolink:ChemicalSubstance', 'is_set': False}}, 'edges': {'e01': {'subject': 'n0', 'object': 'n1', 'predicate': 'biolink:directly_interacts_with', 'relation': None}}}, 'knowledge_graph': {'nodes': {}, 'edges': {}}, 'results': []}}, 'line_number': 67}


In [27]:
aq={'message': {'query_graph': {'nodes': {'n0': {'category': 'biolink:Gene', 'is_set': False, 'id': 'NCBIGene:6611'}, 'n1': {'category': 'biolink:ChemicalSubstance', 'is_set': False}}, 'edges': {'e01': {'subject': 'n0', 'object': 'n1','predicate':'biolink:related_to'}}}}}

In [30]:
print_json(aq)

{
    "message": {
        "query_graph": {
            "nodes": {
                "n0": {
                    "category": "biolink:Gene",
                    "is_set": false,
                    "id": "NCBIGene:6611"
                },
                "n1": {
                    "category": "biolink:ChemicalSubstance",
                    "is_set": false
                }
            },
            "edges": {
                "e01": {
                    "subject": "n0",
                    "object": "n1",
                    "predicate": "biolink:related_to"
                }
            }
        }
    }
}


In [29]:
ar

{'message': {'query_graph': {'nodes': {'n0': {'id': 'NCBIGene:6611',
     'category': 'biolink:Gene',
     'is_set': False},
    'n1': {'id': None,
     'category': 'biolink:ChemicalSubstance',
     'is_set': False}},
   'edges': {'e01': {'subject': 'n0',
     'object': 'n1',
     'predicate': 'biolink:related_to',
     'relation': None}}},
  'knowledge_graph': {'nodes': {}, 'edges': {}},
  'results': []}}

In [43]:
res = print_query_for_source(strider_result,'https://stars-app.renci.org/cam-kp/query')

{'message': 'Received response from KP', 'url': 'https://stars-app.renci.org/cam-kp/query', 'request': {'message': {'query_graph': {'nodes': {'n0': {'category': 'biolink:Gene', 'is_set': False, 'id': 'UniProtKB:P52788'}, 'n1': {'category': 'biolink:ChemicalSubstance', 'is_set': False}}, 'edges': {'e01': {'subject': 'n0', 'object': 'n1', 'predicate': 'biolink:affects'}}}}}, 'response': {'message': {'query_graph': {'nodes': {'n0': {'id': 'UniProtKB:P52788', 'category': 'biolink:Gene', 'is_set': False}, 'n1': {'category': 'biolink:ChemicalSubstance', 'is_set': False}}, 'edges': {'e01': {'subject': 'n0', 'object': 'n1', 'predicate': 'biolink:affects'}}}, 'knowledge_graph': {'nodes': {'http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-ALL-1237175': {'name': '2-Oxoacid', 'category': ['biolink:NamedThing', 'biolink:BiologicalEntity', 'biolink:MolecularEntity', 'biolink:ChemicalSubstance']}, 'http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-ALL-174390': {'name': 

In [50]:
res['request']['message']

{'query_graph': {'nodes': {'n0': {'category': 'biolink:Gene',
    'is_set': False,
    'id': 'UniProtKB:P52788'},
   'n1': {'category': 'biolink:ChemicalSubstance', 'is_set': False}},
  'edges': {'e01': {'subject': 'n0',
    'object': 'n1',
    'predicate': 'biolink:affects'}}}}

In [55]:
for r in res['response']['message']['results']:
    print (r['node_bindings'])

{'n0': [{'id': 'http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-351206'}], 'n1': [{'id': 'http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-1237095'}]}
{'n0': [{'id': 'NCBIGENE:6611'}], 'n1': [{'id': 'http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-1237095'}]}
{'n0': [{'id': 'UBERON:0001062'}], 'n1': [{'id': 'http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-1237095'}]}
{'n0': [{'id': 'http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-351206'}], 'n1': [{'id': 'NCBIGENE:84245'}]}
{'n0': [{'id': 'NCBIGENE:6611'}], 'n1': [{'id': 'NCBIGENE:84245'}]}
{'n0': [{'id': 'UBERON:0001062'}], 'n1': [{'id': 'NCBIGENE:84245'}]}
{'n0': [{'id': 'http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-351206'}], 'n1': [{'id': 'UBERON:0001062'}]}
{'n0': [{'id': 'NCBIGENE:6611'}], 'n1': [{'id': 'UBERON:0001062'}]}
{'n0': [{'id': 'UBERON:0001062'}], 'n1': [{'id': 'UBERON:0001062'}]}


## ARAGORN 

In [19]:
start = dt.now()
aragorn_result = aragorn(query)
end = dt.now()
if 'results' in aragorn_result['message']:
    print(f"ARAGORN produced {len(aragorn_result['message']['results'])} results in {end-start}.")
else:
    print('Error, no result field')

ARAGORN produced 37 results in 0:01:07.387003.


In [20]:
view = GammaViewer(props={"data":aragorn_result})
display(view)

### ARAGORN Assessment

How did we do?